# Quora Question Pairs using MaLSTM 

In [ ]:
!pip install fuzzywuzzy distance

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
import numpy as np

# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
#from nltk.corpus import stopword
from nltk.stem import WordNetLemmatizer
import re


import seaborn as sns
# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
#from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import distance

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
import tensorflow as tf
from keras.engine.topology import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Embedding,Bidirectional,LSTM,Attention,Concatenate,GlobalAveragePooling1D,Dropout, Lambda
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

train_df = pd.read_csv( "/content/drive/MyDrive/Kaggle/train_preprocessed.csv") 
embeddings_index = pickle.load( open( "/content/drive/MyDrive/Kaggle/embeddings_index.p", "rb" ) )

In [ ]:
max_num_words = 200000
sequence_length = 60
embedding_dim = 300

In [ ]:
train_df['question1_clean']=train_df['question1_clean'].astype(str)
train_df['question2_clean']=train_df['question2_clean'].astype(str)

In [ ]:
train_df.describe()

,id,qid1,qid2,is_duplicate,q1len,q2len,q1_n_words,q2_n_words,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,word_share
count,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198,59.536709,60.108365,10.944592,11.185120,64.058594,61.951198,64.936976,0.370069,0.238862
std,116708.614502,157751.700002,159903.182629,0.482588,29.940655,33.863870,5.431949,6.311076,16.885971,18.374723,16.797087,0.224148,0.128373
min,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000,39.000000,39.000000,7.000000,7.000000,52.000000,47.000000,51.000000,0.195122,0.142857
50%,202144.500000,192182.000000,197052.000000,0.000000,52.000000,51.000000,10.000000,10.000000,63.000000,61.000000,64.000000,0.325000,0.235294
75%,303216.750000,346573.500000,354692.500000,1.000000,72.000000,72.000000,13.000000,13.000000,77.000000,77.000000,78.000000,0.500000,0.333333
max,404289.000000,537932.000000,537933.000000,1.000000,623.000000,1169.000000,125.000000,237.000000,100.000000,100.000000,100.000000,0.994186,0.500000


In [ ]:
q1 = train_df['question1_clean'].values
q2 = train_df['question2_clean'].values

tokenizer = Tokenizer(num_words = max_num_words, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(q1+q2)

# this takes our sentences and replaces each word with an integer
X_1 = tokenizer.texts_to_sequences(q1)
X_2 = tokenizer.texts_to_sequences(q2)

# we then pad the sequences so they're all the same length (sequence_length)
X_1 = pad_sequences(X_1, int(sequence_length))
X_2 = pad_sequences(X_2, int(sequence_length))

y = train_df['is_duplicate']

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 86392 unique tokens.


In [ ]:
# Building embedding_matrix usig embeddings_index
embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))


Null word embeddings: 


In [ ]:
q1_n_words=train_df['q1_n_words'].values
q2_n_words=train_df['q2_n_words'].values
have_same_q_w=train_df['have_same_q_w'].values

In [ ]:
# Standarize numerical features 
scaler = MinMaxScaler()
scaler.fit(q1_n_words.reshape(-1,1))
q1_n_words=scaler.transform(q1_n_words.reshape(-1,1))

scaler.fit(q2_n_words.reshape(-1,1))
q2_n_words=scaler.transform(q2_n_words.reshape(-1,1))

scaler.fit(have_same_q_w.reshape(-1,1))
have_same_q_w=scaler.transform(have_same_q_w.reshape(-1,1))


In [ ]:
def exponent_neg_manhattan_distance(left, right):
    ''' 
    Purpose : Helper function for the similarity estimate of the LSTMs outputs
    Inputs : Two n-dimensional vectors
    Output : Manhattan distance between the input vectors
    
    '''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [ ]:
# Input layers
q1_input = Input(shape=(sequence_length,))
q2_input = Input(shape=(sequence_length,))

embedding_layer = Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                             weights=[embedding_matrix],input_length = sequence_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(q1_input)
encoded_right = embedding_layer(q2_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(50)

q1_output = shared_lstm(encoded_left)
q2_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
output = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_output, q2_output])

In [ ]:
# Combine all of the above in a Model
model = Model([q1_input, q2_input], [output])

In [ ]:
model.compile(loss='mse', optimizer= 'adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      60000000    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 60)           86640       embedding_1[0][0]          

In [ ]:
history = model.fit([X_1, X_2], y, epochs = 100, batch_size = 2048, verbose = 1, validation_split = 0.1, shuffle = True)

Epoch 1/100
178/178 [==============================] - 22s 108ms/step - loss: 0.2403 - accuracy: 0.6826 - val_loss: 0.1666 - val_accuracy: 0.7562
Epoch 2/100
178/178 [==============================] - 18s 104ms/step - loss: 0.1659 - accuracy: 0.7565 - val_loss: 0.1562 - val_accuracy: 0.7753
Epoch 3/100
178/178 [==============================] - 18s 104ms/step - loss: 0.1561 - accuracy: 0.7751 - val_loss: 0.1503 - val_accuracy: 0.7854
Epoch 4/100
178/178 [==============================] - 18s 103ms/step - loss: 0.1500 - accuracy: 0.7875 - val_loss: 0.1465 - val_accuracy: 0.7921
Epoch 5/100
178/178 [==============================] - 18s 102ms/step - loss: 0.1466 - accuracy: 0.7932 - val_loss: 0.1437 - val_accuracy: 0.7972
Epoch 6/100
178/178 [==============================] - 18s 103ms/step - loss: 0.1430 - accuracy: 0.7996 - val_loss: 0.1425 - val_accuracy: 0.8025
Epoch 7/100
178/178 [==============================] - 18s 103ms/step - loss: 0.1406 - accuracy: 0.8047 - val_loss: 0.1399 -

In [2]:
# # list all data in history
# print(history.history.keys())

# # summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

NameError: ignored